In [ ]:
import os
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import plotly.graph_objects as go
import wandb
import numpy as np
from wandb.integration.xgboost import WandbCallback

In [ ]:
# Initialize wandb run
# Expanded parameter dictionary for production-style training visibility
param = {
    # Objective and evaluation
    "objective": "binary:logistic",
    "eval_metric": ["aucpr", "logloss"],
    # Capacity and regularization
    "n_estimators": 2000,  # training budget (used with early stopping)
    "eta": 0.03,  # alias: learning_rate
    "max_depth": 6,
    "min_child_weight": 3,
    "gamma": 0.0,  # min loss reduction to split
    "reg_lambda": 1.0,  # L2 regularization
    "reg_alpha": 0.0,  # L1 regularization
    # Sampling for generalization
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "colsample_bylevel": 1.0,
    "colsample_bynode": 1.0,
    # Tree construction
    "booster": "gbtree",
    "tree_method": "hist",  # use "gpu_hist" if GPU is available
    "grow_policy": "depthwise",  # consider "lossguide" for very large data
    "max_bin": 256,
    # Class imbalance and constraints
    "scale_pos_weight": 1.0,  # set to (neg/pos) if imbalanced
    "monotone_constraints": {},  # e.g., {"feature_name": 1/-1}
    "interaction_constraints": [],
    # Training controls
    "early_stopping_rounds": 100,
    "verbosity": 1,
    "seed": 42,
    "nthread": 4,  # alias: n_jobs
}

run = wandb.init(
    project="book-recommendation",
    group="dev",
    job_type="train",
    save_code=True,
    config=param,
)

In [ ]:
artifact = run.use_artifact("book-recommendation/completion_prediction.csv:latest")
artifact_path = artifact.file()

In [ ]:
df = pd.read_csv(artifact_path)

print("Completion prediction model")
print("Task: Will user complete books they interact with?")
print(f"Dataset shape: {df.shape}")
print("Target: is_read (completion)")
print(f"Features: {df.shape[1] - 1}")

# Check target distribution
print("\nTarget distribution:")
print(df["is_read"].value_counts())
completion_rate = df["is_read"].mean()
print(f"Overall completion rate: {completion_rate:.3f}")

In [ ]:
df.head()

In [ ]:
# Prepare data
X = df.iloc[:, 1:]
y = df.iloc[:, 0]
sz = df.shape

# simple sequential split to keep any temporal order (if present)
test_size = 0.2
seed = 42
split_idx = int(sz[0] * (1 - test_size))

X_train = X.iloc[:split_idx, :]
X_test = X.iloc[split_idx:, :]
y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

# Validation split from training (do not use test for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=seed
)

# Mirror key params into W&B (optional, param is already in wandb.config)
wandb.config["data_shape"] = sz

print("Clean completion prediction setup with early stopping (from param)")
print("No leaky features (rating, review_length removed)")
print("No synthetic negatives")
print("Focus: Will user complete books they choose to interact with?")

# Map param dict to sklearn API
eval_metric = param.get("eval_metric", ["aucpr", "logloss"])
if isinstance(eval_metric, str):
    eval_metric_list = [eval_metric]
else:
    eval_metric_list = list(eval_metric)

# Early stopping settings from param
early_rounds = int(param.get("early_stopping_rounds", 100))

# Choose metric to early stop on (prefer aucpr if present)
if "aucpr" in eval_metric_list:
    es_metric = "aucpr"
else:
    es_metric = eval_metric_list[0] if len(eval_metric_list) > 0 else "logloss"
maximize = es_metric not in [
    "rmse",
    "mae",
    "logloss",
    "merror",
    "mlogloss",
    "poisson-nloglik",
]


callbacks = [
    WandbCallback(),
    xgb.callback.EarlyStopping(
        rounds=early_rounds,
        metric_name=es_metric,
        data_name="validation_1",
        save_best=True,
        maximize=maximize,
    ),
]

xgb_params = {
    "objective": param.get("objective", "binary:logistic"),
    "n_estimators": int(param.get("n_estimators", 2000)),
    "learning_rate": float(param.get("eta", 0.03)),
    "max_depth": int(param.get("max_depth", 6)),
    "min_child_weight": float(param.get("min_child_weight", 3)),
    "gamma": float(param.get("gamma", 0.0)),
    "reg_lambda": float(param.get("reg_lambda", 1.0)),
    "reg_alpha": float(param.get("reg_alpha", 0.0)),
    "subsample": float(param.get("subsample", 0.8)),
    "colsample_bytree": float(param.get("colsample_bytree", 0.8)),
    "colsample_bylevel": float(param.get("colsample_bylevel", 1.0)),
    "colsample_bynode": float(param.get("colsample_bynode", 1.0)),
    "booster": param.get("booster", "gbtree"),
    "tree_method": param.get("tree_method", "hist"),
    "grow_policy": param.get("grow_policy", "depthwise"),
    "max_bin": int(param.get("max_bin", 256)),
    "scale_pos_weight": float(param.get("scale_pos_weight", 1.0)),
    "n_jobs": int(param.get("nthread", -1)),
    "random_state": int(param.get("seed", seed)),
    "eval_metric": eval_metric_list,
    "callbacks": callbacks,
}

cls = xgb.XGBClassifier(**xgb_params)
cls.fit(
    X_tr,
    y_tr,
    eval_set=[(X_tr, y_tr), (X_val, y_val)],
    verbose=False,
)

# get prediction on held-out test
pred = cls.predict(X_test)
error_rate = (pred != y_test).mean()

# log metrics to wandb
wandb.summary["Error Rate"] = float(error_rate)
# best iteration and score
bst = cls.get_booster()
if hasattr(bst, "best_iteration") and bst.best_iteration is not None:
    run.summary["best_iteration"] = int(bst.best_iteration)
ev = cls.evals_result()
if "validation_1" in ev and es_metric in ev["validation_1"]:
    run.summary["best_" + es_metric] = float(
        max(ev["validation_1"][es_metric])
        if maximize
        else min(ev["validation_1"][es_metric])
    )

In [ ]:
preds = cls.predict(X_test)

In [ ]:
# Basic classification metrics
import numpy as np
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    average_precision_score,
)
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("Basic classification metrics")
print(f"Test Accuracy: {accuracy_score(y_test, preds):.4f}")
print(f"Precision: {precision_score(y_test, preds):.4f}")
print(f"Recall: {recall_score(y_test, preds):.4f}")
print(f"F1-Score: {f1_score(y_test, preds):.4f}")

# Get prediction probabilities for ranking
y_proba = cls.predict_proba(X_test)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}")
print(f"PR-AUC: {average_precision_score(y_test, y_proba):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, preds))

# Confusion matrix with plotly
cm = confusion_matrix(y_test, preds)
print("\nConfusion Matrix:")
print(cm)

# Create confusion matrix heatmap
fig_cm = go.Figure(
    data=go.Heatmap(
        z=cm,
        x=["Predicted 0", "Predicted 1"],
        y=["Actual 0", "Actual 1"],
        colorscale="Blues",
        text=cm,
        texttemplate="%{text}",
        textfont={"size": 20},
        showscale=True,
    )
)

fig_cm.update_layout(
    title="Confusion Matrix",
    xaxis_title="Predicted Label",
    yaxis_title="Actual Label",
    width=500,
    height=400,
)

fig_cm.show()
run.log({"confusion_matrix": fig_cm})

In [ ]:
# ROC and precision-recall curves

# Calculate curves
fpr, tpr, roc_thresholds = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

precision, recall, pr_thresholds = precision_recall_curve(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)

# Create subplots for ROC and PR curves
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=["ROC Curve", "Precision-Recall Curve"],
    specs=[[{"secondary_y": False}, {"secondary_y": False}]],
)

# ROC curve
fig.add_trace(
    go.Scatter(
        x=fpr,
        y=tpr,
        mode="lines",
        name=f"ROC Curve (AUC = {roc_auc:.3f})",
        line=dict(color="blue", width=2),
    ),
    row=1,
    col=1,
)

# Diagonal line for ROC
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode="lines",
        name="Random Classifier",
        line=dict(color="red", width=1, dash="dash"),
        showlegend=False,
    ),
    row=1,
    col=1,
)

# Precision-recall curve
fig.add_trace(
    go.Scatter(
        x=recall,
        y=precision,
        mode="lines",
        name=f"PR Curve (AUC = {pr_auc:.3f})",
        line=dict(color="green", width=2),
    ),
    row=1,
    col=2,
)

# Baseline for PR curve
baseline = y_test.sum() / len(y_test)  # Positive rate
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[baseline, baseline],
        mode="lines",
        name=f"Random Baseline ({baseline:.3f})",
        line=dict(color="red", width=1, dash="dash"),
        showlegend=False,
    ),
    row=1,
    col=2,
)

# Update layout
fig.update_xaxes(title_text="False Positive Rate", row=1, col=1)
fig.update_yaxes(title_text="True Positive Rate", row=1, col=1)
fig.update_xaxes(title_text="Recall", row=1, col=2)
fig.update_yaxes(title_text="Precision", row=1, col=2)

fig.update_layout(
    title="Model Performance Curves", width=1000, height=400, showlegend=True
)

fig.show()
run.log({"roc_pr_curves": fig})

print("Model performance summary:")
print(
    f"ROC-AUC: {roc_auc:.4f} ({'Excellent' if roc_auc > 0.9 else 'Good' if roc_auc > 0.8 else 'Fair' if roc_auc > 0.7 else 'Poor'})"
)
print(
    f"PR-AUC: {pr_auc:.4f} ({'Excellent' if pr_auc > 0.8 else 'Good' if pr_auc > 0.6 else 'Fair' if pr_auc > 0.4 else 'Poor'})"
)
print(f"Baseline (random): {baseline:.4f}")

In [ ]:
# Prediction distribution analysis

# Create prediction distribution plots
fig_dist = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=[
        "Prediction Probability Distribution",
        "Prediction Probabilities by Class",
    ],
    specs=[[{"secondary_y": False}, {"secondary_y": False}]],
)

# Overall distribution
fig_dist.add_trace(
    go.Histogram(
        x=y_proba,
        nbinsx=50,
        name="All Predictions",
        opacity=0.7,
        marker_color="lightblue",
    ),
    row=1,
    col=1,
)

# Distribution by class
fig_dist.add_trace(
    go.Histogram(
        x=y_proba[y_test == 0],
        nbinsx=30,
        name="Negative Class (0)",
        opacity=0.7,
        marker_color="red",
    ),
    row=1,
    col=2,
)

fig_dist.add_trace(
    go.Histogram(
        x=y_proba[y_test == 1],
        nbinsx=30,
        name="Positive Class (1)",
        opacity=0.7,
        marker_color="green",
    ),
    row=1,
    col=2,
)

fig_dist.update_xaxes(title_text="Prediction Probability", row=1, col=1)
fig_dist.update_yaxes(title_text="Count", row=1, col=1)
fig_dist.update_xaxes(title_text="Prediction Probability", row=1, col=2)
fig_dist.update_yaxes(title_text="Count", row=1, col=2)

fig_dist.update_layout(
    title="Prediction Probability Distributions",
    width=1000,
    height=400,
    showlegend=True,
    barmode="overlay",  # For overlapping histograms in the second subplot
)

fig_dist.show()
run.log({"prediction_distributions": fig_dist})

# Calculate separation metrics
mean_pos = y_proba[y_test == 1].mean()
mean_neg = y_proba[y_test == 0].mean()
separation = abs(mean_pos - mean_neg)

print("Prediction analysis:")
print(f"Mean probability for positive class: {mean_pos:.4f}")
print(f"Mean probability for negative class: {mean_neg:.4f}")
print(
    f"Class separation: {separation:.4f} ({'Good' if separation > 0.3 else 'Moderate' if separation > 0.1 else 'Poor'})"
)
print("Optimal threshold (balanced): 0.5")

In [ ]:
# Load completion prediction mappings
user_book_mapping = pd.read_csv("../data/completion_user_book_mapping.csv")

print("=== Completion Prediction Evaluation ===")
print("Task: Predicting if users complete books they interact with")
print(f"Mapping shape: {user_book_mapping.shape}")

# Create test mapping for completion prediction
test_indices = X_test.index
test_mapping = user_book_mapping.iloc[test_indices].copy()
test_mapping["prediction_score"] = y_proba
test_mapping["true_completion"] = y_test.values  # is_read target

print(f"Test mapping created: {len(test_mapping)} samples")
print(f"Users in test: {test_mapping['user_id'].nunique()}")
print(f"Books completed in test: {test_mapping['true_completion'].sum()}")

test_mapping.head()

In [ ]:
# Ranking evaluation functions


def compute_precision_at_k(user_data, k):
    """Compute Precision@k for a single user"""
    top_k = user_data.head(k)
    return top_k["true_completion"].sum() / k


def compute_recall_at_k(user_data, k):
    """Compute Recall@k for a single user"""
    top_k = user_data.head(k)
    total_relevant = user_data["true_completion"].sum()
    if total_relevant == 0:
        return 0.0
    return top_k["true_completion"].sum() / total_relevant


def compute_ap_at_k(user_data, k):
    """Compute Average Precision@k for a single user"""
    top_k = user_data.head(k)
    y_true = top_k["true_completion"].values
    y_scores = top_k["prediction_score"].values

    if y_true.sum() == 0:
        return 0.0

    return average_precision_score(y_true, y_scores)


def compute_ndcg_at_k(user_data, k):
    """Compute NDCG@k for a single user"""
    from sklearn.metrics import ndcg_score

    top_k = user_data.head(k)
    y_true = top_k["true_completion"].values.reshape(1, -1)
    y_scores = top_k["prediction_score"].values.reshape(1, -1)

    if len(y_true[0]) == 0:
        return 0.0

    return ndcg_score(y_true, y_scores, k=k)


def evaluate_ranking_metrics(test_mapping, k_values=[5, 10, 20, 50]):
    """Compute comprehensive ranking metrics"""

    metrics = {f"precision@{k}": [] for k in k_values}
    metrics.update({f"recall@{k}": [] for k in k_values})
    metrics.update({f"map@{k}": [] for k in k_values})
    metrics.update({f"ndcg@{k}": [] for k in k_values})

    users_evaluated = 0

    for user_id in test_mapping["user_id"].unique():
        user_data = test_mapping[test_mapping["user_id"] == user_id]

        # Skip users with no positive items in test
        if user_data["true_completion"].sum() == 0:
            continue

        # Sort by prediction score (highest first)
        user_data = user_data.sort_values("prediction_score", ascending=False)
        users_evaluated += 1

        # Compute metrics for each k
        for k in k_values:
            if len(user_data) >= k:  # Only compute if user has enough items
                metrics[f"precision@{k}"].append(compute_precision_at_k(user_data, k))
                metrics[f"recall@{k}"].append(compute_recall_at_k(user_data, k))
                metrics[f"map@{k}"].append(compute_ap_at_k(user_data, k))
                metrics[f"ndcg@{k}"].append(compute_ndcg_at_k(user_data, k))

    # Average across users
    results = {}
    for metric_name, values in metrics.items():
        if values:  # Only compute average if we have values
            results[metric_name] = np.mean(values)
        else:
            results[metric_name] = 0.0

    results["users_evaluated"] = users_evaluated
    return results


print("Computing ranking metrics")
ranking_results = evaluate_ranking_metrics(test_mapping, k_values=[5, 10, 20, 50])

print(f"Users evaluated: {ranking_results['users_evaluated']}")
print("\nRanking metrics:")
for metric, value in ranking_results.items():
    if metric != "users_evaluated":
        print(f"{metric.upper()}: {value:.4f}")
# Log ranking metrics to Weights & Biases
_rank_metrics = {"ranking/users_evaluated": ranking_results.get("users_evaluated", 0)}
for _k in [5, 10, 20, 50]:
    _rank_metrics.update(
        {
            f"ranking/precision@{_k}": float(
                ranking_results.get(f"precision@{_k}", 0.0)
            ),
            f"ranking/recall@{_k}": float(ranking_results.get(f"recall@{_k}", 0.0)),
            f"ranking/map@{_k}": float(ranking_results.get(f"map@{_k}", 0.0)),
            f"ranking/ndcg@{_k}": float(ranking_results.get(f"ndcg@{_k}", 0.0)),
        }
    )
run.log(_rank_metrics)

# Also log a compact table
rank_table = wandb.Table(columns=["k", "precision", "recall", "map", "ndcg"])
for _k in [5, 10, 20, 50]:
    rank_table.add_data(
        _k,
        float(ranking_results.get(f"precision@{_k}", 0.0)),
        float(ranking_results.get(f"recall@{_k}", 0.0)),
        float(ranking_results.get(f"map@{_k}", 0.0)),
        float(ranking_results.get(f"ndcg@{_k}", 0.0)),
    )
run.log({"ranking/metrics_table": rank_table})

# Promote commonly tracked ones to summary
for _key in ["precision@20", "recall@20", "map@20", "ndcg@20"]:
    run.summary[f"ranking/{_key}"] = float(ranking_results.get(_key, 0.0))

In [ ]:
# Feature importance analysis

print("Feature importance analysis")

# Get feature importance from XGBoost
feature_importance = cls.feature_importances_
feature_names = X.columns

# Create importance DataFrame
importance_df = pd.DataFrame(
    {"feature": feature_names, "importance": feature_importance}
).sort_values("importance", ascending=False)

print("Top 15 most important features:")
print(importance_df.head(15))

# Plot feature importance with Plotly
top_features = importance_df.head(20)

fig = go.Figure(
    go.Bar(
        x=top_features["importance"],
        y=top_features["feature"],
        orientation="h",
        marker_color="lightblue",
        text=top_features["importance"].round(4),
        textposition="auto",
    )
)

fig.update_layout(
    title="Top 20 Feature Importances (XGBoost)",
    xaxis_title="Feature Importance",
    yaxis_title="Features",
    height=600,
    width=900,
    yaxis={"categoryorder": "total ascending"},
    showlegend=False,
)

fig.show()
run.log({"feature_importance": fig})

# Save feature importance
importance_df.to_csv("../data/feature_importance.csv", index=False)

In [ ]:
# Performance report

print("=" * 80)
print("BOOK RECOMMENDATION SYSTEM - PERFORMANCE REPORT")
print("=" * 80)

print(f"""
DATASET OVERVIEW:
   • Total samples: {len(df):,}
   • Training samples: {len(X_train):,}
   • Test samples: {len(X_test):,}
   • Features: {X.shape[1]}
   • Unique users in test: {test_mapping["user_id"].nunique():,}
   • Users evaluated for ranking: {ranking_results["users_evaluated"]:,}

CLASSIFICATION PERFORMANCE:
   • Accuracy: {accuracy_score(y_test, preds):.4f}
   • Precision: {precision_score(y_test, preds):.4f}
   • Recall: {recall_score(y_test, preds):.4f}
   • F1-Score: {f1_score(y_test, preds):.4f}
   • ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}

RANKING PERFORMANCE:""")

# Format ranking metrics nicely
metrics_by_k = {}
for k in [5, 10, 20, 50]:
    metrics_by_k[k] = {
        "Precision": ranking_results[f"precision@{k}"],
        "Recall": ranking_results[f"recall@{k}"],
        "mAP": ranking_results[f"map@{k}"],
        "NDCG": ranking_results[f"ndcg@{k}"],
    }

print("   ┌─────────┬──────────┬─────────┬─────────┬─────────┐")
print("   │    k    │ Precision│  Recall │   mAP   │  NDCG   │")
print("   ├─────────┼──────────┼─────────┼─────────┼─────────┤")
for k in [5, 10, 20, 50]:
    metrics = metrics_by_k[k]
    print(
        f"   │   @{k:2d}   │  {metrics['Precision']:.4f}  │ {metrics['Recall']:.4f}  │ {metrics['mAP']:.4f}  │ {metrics['NDCG']:.4f}  │"
    )
print("   └─────────┴──────────┴─────────┴─────────┴─────────┘")

print("""
TOP PREDICTIVE FEATURES:""")
for i, (feature, importance) in enumerate(importance_df.head(10).values, 1):
    print(f"   {i:2d}. {feature:<25} ({importance:.4f})")

print(f"""
RECOMMENDATION QUALITY INSIGHTS:
   • mAP@20 of {ranking_results["map@20"]:.4f} indicates {"excellent" if ranking_results["map@20"] > 0.3 else "good" if ranking_results["map@20"] > 0.1 else "moderate"} ranking quality
   • Precision@20 of {ranking_results["precision@20"]:.4f} means {ranking_results["precision@20"] * 100:.1f}% of top-20 recommendations are relevant
   • Model successfully learns from {X.shape[1]} engineered features
   • {importance_df.head(5)["feature"].str.contains("similarity").sum()} of top-5 features are similarity-based

BUSINESS IMPACT:
   • For every 20 books recommended, ~{ranking_results["precision@20"] * 20:.0f} will be relevant to the user
   • {ranking_results["recall@20"] * 100:.1f}% of relevant books are captured in top-20 recommendations
   • Strong classification performance (AUC: {roc_auc_score(y_test, y_proba):.3f}) enables confident ranking
""")

print("=" * 80)

# Save report to file
report_text = f"""Book Recommendation System Performance Report
Generated: {pd.Timestamp.now()}

Dataset Overview:
- Total samples: {len(df):,}
- Training samples: {len(X_train):,}
- Test samples: {len(X_test):,}
- Features: {X.shape[1]}
- Users evaluated: {ranking_results["users_evaluated"]:,}

Classification Metrics:
- Accuracy: {accuracy_score(y_test, preds):.4f}
- Precision: {precision_score(y_test, preds):.4f}
- Recall: {recall_score(y_test, preds):.4f}
- F1-Score: {f1_score(y_test, preds):.4f}
- ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}

Ranking Metrics:
"""

for k in [5, 10, 20, 50]:
    report_text += f"@{k}: Precision={ranking_results[f'precision@{k}']:.4f}, Recall={ranking_results[f'recall@{k}']:.4f}, mAP={ranking_results[f'map@{k}']:.4f}, NDCG={ranking_results[f'ndcg@{k}']:.4f}\n"

report_text += "\nTop 10 Features:\n"
for i, (feature, importance) in enumerate(importance_df.head(10).values, 1):
    report_text += f"{i}. {feature}: {importance:.4f}\n"

# Save to file
with open("../data/model_performance_report.txt", "w") as f:
    f.write(report_text)

print("Detailed report saved to: ../data/model_performance_report.txt")

In [ ]:
# Save model and artifacts

import joblib

print("Saving model and artifacts")

# Save the trained model
model_path = "../models/xgboost_recommender_model.pkl"
joblib.dump(cls, model_path)
print(f"Model saved to: {model_path}")

# Save test results for further analysis
test_results_path = "../data/test_results.csv"
test_mapping.to_csv(test_results_path, index=False)
print(f"Test results saved to: {test_results_path}")

# Save ranking metrics summary
ranking_summary = pd.DataFrame([ranking_results]).T
ranking_summary.columns = ["value"]
ranking_summary_path = "../data/ranking_metrics_summary.csv"
ranking_summary.to_csv(ranking_summary_path)
print(f"Ranking metrics saved to: {ranking_summary_path}")

print(f"""
Training complete.

Generated files:
   • Model: {model_path}
   • Test Results: {test_results_path}
   • Feature Importance: ../data/feature_importance.csv
   • Performance Report: ../data/model_performance_report.txt
   • Ranking Metrics: {ranking_summary_path}
""")

# Save training data split and upload as artifact
train_df = pd.concat([y_train.rename("is_read"), X_train], axis=1)
train_data_filename = "train_data.csv"
train_data_path = os.path.join("..", "data", train_data_filename)
train_df.to_csv(train_data_path, index=False)
print(f"Training data saved to: {train_data_path}")
train_art = wandb.Artifact(
    name=train_data_filename,
    type="training_data",
    description="Book completion prediction training data",
)
train_art.add_file(train_data_path)
run.log_artifact(train_art)

# Upload saved files as artifacts
files_to_upload = [
    (model_path, "model", "XGBoost recommender model"),
    ("../data/feature_importance.csv", "dataset", "XGBoost feature importances"),
    (test_results_path, "dataset", "Test results CSV"),
    (ranking_summary_path, "dataset", "Ranking metrics summary"),
    ("../data/model_performance_report.txt", "report", "Text performance report"),
]
for path, atype, desc in files_to_upload:
    base = os.path.basename(path)
    art = wandb.Artifact(name=os.path.splitext(base)[0], type=atype, description=desc)
    art.add_file(path)
    run.log_artifact(art)

In [ ]:
run.finish()